# TestData for `empymod`

Compare existing code with previously created data. Show fails for given error.

In [1]:
import os
import shutil
import pickle
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from IPython.display import clear_output
from timeit import default_timer as time

from empymod import utils
from empymod.model import dipole

### Plot function

In [2]:
def plotresultabs(one, two, offs, ignoregood, prntstr):
    
    # Calculate Errors
    err = np.log10(np.abs((np.abs(one)-np.abs(two)/np.abs(two))))

    # Set +/- inf to nan, to calc max for non-inf non-nan values
    err[np.abs(err) >= np.finfo(np.double).max ] = np.nan
    
    # Get indices for the four quadrants
    xy0 = (irec[0] != 0)*(irec[1] != 0)  
    xy1 = (irec[0] > 0)*(irec[1] >= 0)    
    xy2 = (irec[0] <= 0)*(irec[1] > 0)    
    xy3 = (irec[0] < 0)*(irec[1] <= 0)   
    xy4 = (irec[0] >= 0)*(irec[1] < 0)
    offs = np.sqrt(irec[0]**2 + irec[1]**2)

    # Return if good; definer here relative and absolute error!
    if ignoregood and (np.isclose(np.abs(one)[xy0], np.abs(two)[xy0], .02, 1e-18)).all():
        return
    
    print(prntstr, '; Max Err :: ', np.round(np.nanmax(err[xy0]), 3), '/', np.round(np.max(err[xy0]), 3))
    
    # Figure
    fig = plt.figure(figsize=(12,2), facecolor='w')
    fig.subplots_adjust(wspace=.3, hspace=0)

    ## ABS ##
    ax1 = plt.subplot(1,2,1)
    plt.grid('off')
    plt.plot(offs[xy1], np.log10(np.abs(one.amp[xy1])), 'o', mec='r', mfc='none', label='modem')
    plt.plot(offs[xy1], np.log10(np.abs(two.amp[xy1])), 'r.', label='emmod')
    plt.plot(offs[xy2], np.log10(np.abs(one.amp[xy2])), 'o', mec='b', mfc='none', label='modem')
    plt.plot(offs[xy2], np.log10(np.abs(two.amp[xy2])), 'b.', label='emmod')
    plt.plot(offs[xy3], np.log10(np.abs(one.amp[xy3])), 'o', mec='k', mfc='none', label='modem')
    plt.plot(offs[xy3], np.log10(np.abs(two.amp[xy3])), 'k.', label='emmod')
    plt.plot(offs[xy4], np.log10(np.abs(one.amp[xy4])), 'o', mec='g', mfc='none', label='modem')
    plt.plot(offs[xy4], np.log10(np.abs(two.amp[xy4])), 'g.', label='emmod')
    plt.ylabel(r'$|\Re|$')

    ## ABS ERROR ##
    ax3 = ax1.twinx()
    ax3.yaxis.tick_right()
    plt.plot(offs[xy1], err[xy1], '*', mec = 'r', mfc='none')
    plt.plot(offs[xy2], err[xy2], '*', mec = 'b', mfc='none')
    plt.plot(offs[xy3], err[xy3], '*', mec = 'k', mfc='none')
    plt.plot(offs[xy4], err[xy4], '*', mec = 'g', mfc='none')

    plt.show()

## Model parameters

### 18 models $\times$ 34 ab's $\times$ 3 frequencies $=$ 1836 cases.

- ab : all 36 (34) possibilities
- Frequencies: $f = 0.01, 1.0, 100.0\,\text{Hz}$
- X-coordinates: -9000:3000:9000 m
- Y-coordinates: -6000:2000:6000 m
- Scaling (Depths and Coordinates): 0.001 (100 Hz); 1.0 (1 Hz); 4.0 (0.01 Hz)
- Models:


| layer   | 13   | 00   | 03   | 05   | 30   | 33   | 35   | 50   | 53   | 55   | 24   | 42   |
|:-------:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| 0       |      | s  r | s    | s    |    r |      |      |    r |      |      |      |      |
| 1       | s    |      |      |      |      |      |      |      |      |      |      |      |
| 2       |      |      |      |      |      |      |      |      |      |      | s    |    r |
| 3       |    r |      |    r |      | s    | s  r | s    |      |    r |      |      |      |
| 4       |      |      |      |      |      |      |      |      |      |      |    r | s    |
| 5       |      |      |      |    r |      |      |    r | s    | s    | s  r |      |      |

    => 13 is a homogenous fullspace
    => for 00, 33, and 55 exist 3 cases: zsrc > zrec, zsrc < zrec, zsrc = zrec

- zsrc: lower layer-boundary - 100 m; if lsrc=lrec: [-100, -200, -150] m
- zrec: lower layer-boundary -  90 m; if lsrc=lrec: [-200, -100, -150] m
- Naming: ab_freq_xdirdftxt_srrr

In [3]:
froot = './compData/'
# Define coordinates
y = np.arange(-3,4)*2000 # np.repeat(np.arange(4)*5000,4)
x = np.arange(-3,4)*3000 # y.reshape(4,-1).ravel('F')

# Define power of frequencies and scaling
fpow = np.array([-2,0,2])
ffac = np.array([4, 1, .001])

# Define src-rec configs
pab = (np.arange(60)+11).reshape(6, 10)[:, :6].ravel()      # All possibilities
#pab = (np.arange(30)+11).reshape(3, 10)[:, :3].ravel()      # EE possibilities
#pab = (np.arange(30,60)+11).reshape(3, 10)[:, 3:6].ravel()  # MM possibilities
#pab = (np.arange(30)+11).reshape(3, 10)[:, 3:6].ravel()     # EM possibilities
#pab = (np.arange(30,60)+11).reshape(3, 10)[:, :3].ravel()   # ME possibilities
print("<ab>'s considered ::\n", pab)

# Define model - src and rec
srcn = [1, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 2, 4]
recn = [3, 0, 0, 0, 3, 5, 0, 3, 3, 3, 5, 0, 3, 5, 5, 5, 4, 2]

# Source and receiver depths
dfsw = 0
dfsrc = [-100, -100, -200, -150]
dfrec = [ -90, -200, -100, -150]

# Define rest of model
depth = np.array([0, 500, 1100, 1800, 2600, 3500])
res   = [1, 3, 10, .01, 100, 1]
aniso  = [.5, 4, 2, 1.5, 3, .25]
epermH = [1, 80, 5, 10, 20, 3]
epermV = [1, 80, 5, 20, 10, 8]
mpermH = [1, 1, 50, 1, 1, 3]
mpermV = [1, 1, 1, 50, 10, 1]
xdirect = True

<ab>'s considered ::
 [11 12 13 14 15 16 21 22 23 24 25 26 31 32 33 34 35 36 41 42 43 44 45 46 51
 52 53 54 55 56 61 62 63 64 65 66]


## Check

In [4]:
# Define if save or load; default = False => load
save = False

# Define HT, HTARG, and OPT
ht='FHT'
htarg = 'key_401_2009'
opt = None

# Total number of models:
toti = str(np.size(srcn)*(np.size(pab)-2)*np.size(fpow))
ni = 0
# Start timer
t1 = time()
if save:
    out = dict()
else:
    out = pickle.load(open(froot+'self_'+ht+'_data.pck', 'rb'))

dfsw = 0
# Loop over different scenarios (src/rec in different layers)
for i in np.arange(np.size(srcn)):
    # Get src and rec layer, store in model
    sr = srcn[i]
    rr = recn[i]
    if dfsw == 3:
        dfsw = 0
    if sr == rr:
        dfsw += 1
        dftxt = str(dfsw)
    else:
        dftxt = ''
        dfsw = 0
    if dfsw == 3:
        xdir = '1'
    else:
        xdir = '0'
    print('           =============== MODEL '+str(sr)+str(rr)+' ===============')
    
    # First run is a fullspace (all parameters are equal)
    # All others are the model as defined
    if sr == 1 and rr == 3:
        ires = np.size(res)*[5,]
        ianiso = np.size(res)*[2,]
        iepermH = np.size(res)*[40,]
        iepermV = np.size(res)*[80,]
        impermH = np.size(res)*[10,]
        impermV = np.size(res)*[20,]
    else:
        ires = res
        ianiso = aniso
        iepermH = epermH
        iepermV = epermV
        impermH = mpermH
        impermV = mpermV

    # Loop over ab-configurations
    for ab in pab:
        if ab in [36,63]:
            continue            
            
        # Loop over frequencies
        for ifr in np.arange(3):

            freq = fpow[ifr] # Freq

            # Update model with frequency
            ifreq = 10**freq

            scl = ffac[ifr] # Scaling factor
            
            isrc = [0, 0, (depth[sr] + dfsrc[dfsw])*scl]
            irec = [np.repeat(x*scl, 7).reshape(-1, 7).ravel('F'), np.repeat(y*scl, 7),
                    (depth[rr]  + dfrec[dfsw])*scl]
 
            # Update counter
            ni += 1
            
            # Get model-name and print for progress
            modname = str(ab)+'_'+str(freq)+'_'+xdir+dftxt+'_'+str(sr)+str(rr)
            prntstr = 'ab '+str(ab)+'; Model '+str(sr)+str(rr)+'; '+str(ifreq)+' Hz'

            # Start timer
            t0 = time()

            # Run model
            one = dipole(src=isrc, rec=irec, depth=depth[:-1]*scl,
                         res=ires, aniso=ianiso, freqtime=ifreq, ab=ab, ht=ht, htarg=htarg, opt=opt,
                         epermH=iepermH, epermV=iepermV, mpermH=impermH, mpermV=impermV,
                         verb=0, xdirect=xdirect)
            one = utils.EMArray(one.real, -one.imag)
            
            if save:
                out[modname] = one
                    
            else:
                two = utils.EMArray(out[modname].real, out[modname].imag)
                plotresultabs(one, two, irec, True, prntstr)

if save:
    pickle.dump(out, open(froot+compare+'_'+ht+'_data.pck', 'xb'), -1)
                
# Finished message
print('Total time :: ', str(timedelta(seconds=time()-t1)))
print('¡ Finished  '+str(ni)+'/'+toti+'!\n')

           =============== MODEL 13 ===============
           =============== MODEL 00 ===============


/home/dwerthmuller/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:1799: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
/home/dwerthmuller/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:1801: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
/home/dwerthmuller/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in greater_equal
/home/dwerthmuller/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in true_divide


           =============== MODEL 00 ===============
           =============== MODEL 00 ===============
           =============== MODEL 03 ===============
           =============== MODEL 05 ===============
           =============== MODEL 30 ===============
           =============== MODEL 33 ===============
           =============== MODEL 33 ===============
           =============== MODEL 33 ===============
           =============== MODEL 35 ===============
           =============== MODEL 50 ===============
           =============== MODEL 53 ===============
           =============== MODEL 55 ===============
           =============== MODEL 55 ===============
           =============== MODEL 55 ===============
           =============== MODEL 24 ===============
           =============== MODEL 42 ===============
Total time ::  0:00:51.546239
¡ Finished  1836/1836!

